# Lecture 3: Spark Streaming

_Spark Streaming_ is an extension of the Spark API that enables scalabe stream processing.

The continous stream of input data can be ingested from many data sources such as **Kafka**, **Amazon s3** or **TCP sockets**. 

The Spark API allows to process data via high-level functions such as *map* and *reduce*. As we are going to see, it is also possible to use dataframe operations. 

Processed data can be exported to an external database and used to make live dashboards or offline analyses, or stored in files, or be used in a further stage of a Kafka pipeline. 

Spark streaming works by dividing the input data into _micro-batches_ that can be treated as static datasets. In Spark this is referred to as a *discretized stream* (*DStream*). The DStream is represented using RDDs.

![DStream](imgs/lecture3/DStream.png)

Any transformation applied on the DStream, i.e. anything like a `Dstream.map()`, will act independently on each batch. For example, in the image below, we can filter the original RDD to remove some data and produce a new stream. 

![DStream_filter](imgs/lecture3/Dstream_filter.png)

In this lecture we will see how to setup a simple stream using a TCP socket as a data source.

## Setup

In [7]:
# set this variable with one of the following values
# -> 'local'
# -> 'docker_container'
# -> 'docker_cluster'
CLUSTER_TYPE ='docker_cluster'

In [ ]:
# set env variable
%env CLUSTER_TYPE $CLUSTER_TYPE

## Start the cluster 

Environment variables need to be set only in the case of a local cluster

In [ ]:
if CLUSTER_TYPE=='local':
    import findspark
    findspark.init('/home/pazzini/work/courses/MAPD_B/MAPD-B/spark/spark-3.2.1-bin-hadoop3.2')

In [ ]:
%%script bash --no-raise-error

if [[ "$CLUSTER_TYPE" != "docker_cluster" ]]; then
    echo "Launching master and worker"
    
    # start master 
    $SPARK_HOME/sbin/start-master.sh --host localhost \
        --port 7077 --webui-port 8080
    
    # start worker
    $SPARK_HOME/sbin/start-worker.sh spark://localhost:7077 \
        --cores 4 --memory 2g
fi

## Create the Spark session

Later on we will explain what is the role of [Apache Arrow](), but first we need to install it and create the spark session with it.

In [2]:
from pyspark.sql import SparkSession

if CLUSTER_TYPE in ['local', 'docker_container']:
    
    spark = SparkSession.builder \
        .master("spark://localhost:7077")\
        .appName("Spark streaming application")\
        .config("spark.sql.execution.arrow.pyspark.enabled", "true")\
        .config("spark.sql.execution.arrow.pyspark.fallback.enabled", "false")\
        .getOrCreate()

elif CLUSTER_TYPE == 'docker_cluster':
    
    # use the provided master
    spark = SparkSession.builder \
        .master("spark://spark-master:7077")\
        .appName("Spark streaming application")\
        .config("spark.executor.memory", "512m")\
        .config("spark.sql.execution.arrow.pyspark.enabled", "true")\
        .config("spark.sql.execution.arrow.pyspark.fallback.enabled", "false")\
        .getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/05/17 12:46:33 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
sc = spark.sparkContext
sc

<SparkContext master=spark://spark-master:7077 appName=Spark streaming application>

## Spark _Streaming_ context

The first step of a Spark streaming application is the creation of a `StreamingContext`. 

It is a similar concept to the `sparkContext` but it requires to be initialized with some additional information on how to handle the micro-batches.

The context can be initialized using `StreamingContext(SparkContext, batch_interval`). 
The `batch_interval` parameter represents the wall-time between two batches, i.e. the batch duration in seconds.

There could be at most one `StreamingContext` for each Spark application. 

The processing start when `StreamingContext.start()` is called, and it can be stopped with `StreamingContext.stop()`. 

By default, if the `StreamingContext` is stoped without passing the `stopSparkContext=False` option, the sparkContext is also stopped (thus the application is closed).

In [4]:
from pyspark.streaming import StreamingContext

# create a streaming context with a batch interval of 2 seconds
ssc = StreamingContext(sc, 2) 

For this example spark will read data from a TCP socket. 

A dummy data stream is generated by a simple python program that can be found in `utils/producer.py`. 
When executed, the producer will write data on port `5555` of `localhost`. 

The dataset consists of fake credit card transactions.

Have a look at the code from the producer before executing it.

In [5]:
! cat utils/producer.py

import socket
import json
import time
import random
import argparse

first_names=('John','Andy','Joe','Alice')
last_names=('Johnson','Smith','Jones', 'Millers')

def send_messages(client_socket):
    try:
        while 1:
            msg = {
                'name': random.choice(first_names),
                'surname': random.choice(last_names),
                'amount': '{:.2f}'.format(random.random()*1000),
                'delta_t': '{:.2f}'.format(random.random()*10),
                'flag': random.choices([0,1], weights=[0.8, 0.2])[0]
            }
            client_socket.send((json.dumps(msg)+"\n").encode('utf-8'))
            time.sleep(0.1)

    except KeyboardInterrupt:
        exit()

if __name__ == "__main__":

    parser = argparse.ArgumentParser()
    parser.add_argument('--hostname', type=str, required=True)
    args = parser.parse_args()
    print('Using hostname:', args.hostname)

    new_skt = socket.socket()
    host = args.hostname
    port = 5555 
    new_skt.bind

The producer will generate new records in the form of a random combination of:
- firstname
- lastname
- amount
- delta time
- flag

These information will be formatted into a .json data format

To declare to Spark that the StreamingContext data source will be a TCP socket, located at a given `hostname` and `port`, we can use the following 

`socketTextStream(hostname, port)`

Have a look at the StreamingContext documentation to see all the available options, at the [link](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.streaming.StreamingContext.html)

In [6]:
# create a socket stream from the hostname where we collect data from
if CLUSTER_TYPE in ['local', 'docker_container']:
    hostname = "localhost"
    
elif CLUSTER_TYPE == 'docker_cluster':
    hostname = "spark-master"

# and set the port to 5555
socket_stream = ssc.socketTextStream(hostname, 5555)

#### start the python producer.py script

Depending on the Spark deployment mode for this exercise, do either of the following:

**Local** 
- from a terminal, run the python script with the option `--hostname localhost`


**Single Docker container**
- from a terminal, identify the id of the container using `docker ps`
- connect to the Docker container using the `docker exec -it <CONTAINER_ID> bash` command
- from the docker container, run the python script with the option `--hostname localhost`
  
**Docker cluster**
- from a terminal, identify the id of the `spark-master` container using `docker ps`
- connect to the Docker container using the `docker exec -it <CONTAINER_ID> bash` command
- from the docker container, run the python script with the option `--hostname spark-master`

The first thing we need to to is load the json describing each transaction.

In [8]:
import json

# use the map() transformation to apply the same function to all rdds
# the function we want to run is the json.loads() of the messages
json_stream = socket_stream.map(lambda msg: json.loads(msg))

It is possible to print some elements of each batch with `pprint()`. This can be used to explore the RDDs.

In [9]:
json_stream.pprint()

Start the computations with `ssc.start()` and stop with `ssc.stop(stopSparkContext=False)`. Remember that after this last instruction the streaming context must be defined again.

In [10]:
ssc.start()

-------------------------------------------
Time: 2022-05-17 13:11:48
-------------------------------------------
{'name': 'Joe', 'surname': 'Smith', 'amount': '47.03', 'delta_t': '9.20', 'flag': 1}
{'name': 'Andy', 'surname': 'Jones', 'amount': '29.23', 'delta_t': '6.18', 'flag': 0}
{'name': 'Alice', 'surname': 'Jones', 'amount': '428.95', 'delta_t': '9.11', 'flag': 0}
{'name': 'Andy', 'surname': 'Smith', 'amount': '814.78', 'delta_t': '2.03', 'flag': 1}



-------------------------------------------
Time: 2022-05-17 13:11:50
-------------------------------------------
{'name': 'Alice', 'surname': 'Jones', 'amount': '116.99', 'delta_t': '6.78', 'flag': 0}
{'name': 'Andy', 'surname': 'Jones', 'amount': '209.58', 'delta_t': '8.10', 'flag': 0}
{'name': 'Alice', 'surname': 'Smith', 'amount': '951.49', 'delta_t': '6.77', 'flag': 0}
{'name': 'Andy', 'surname': 'Jones', 'amount': '106.78', 'delta_t': '7.08', 'flag': 0}
{'name': 'Joe', 'surname': 'Millers', 'amount': '854.42', 'delta_t': '4.88', 'flag': 0}
{'name': 'Alice', 'surname': 'Jones', 'amount': '359.85', 'delta_t': '6.68', 'flag': 0}
{'name': 'Joe', 'surname': 'Johnson', 'amount': '36.19', 'delta_t': '3.62', 'flag': 0}
{'name': 'John', 'surname': 'Johnson', 'amount': '392.46', 'delta_t': '6.41', 'flag': 0}
{'name': 'Alice', 'surname': 'Smith', 'amount': '639.38', 'delta_t': '3.58', 'flag': 0}
{'name': 'Alice', 'surname': 'Millers', 'amount': '255.69', 'delta_t': '5.64', 'flag': 0}
...



-------------------------------------------
Time: 2022-05-17 13:11:52
-------------------------------------------
{'name': 'Joe', 'surname': 'Johnson', 'amount': '378.04', 'delta_t': '9.16', 'flag': 0}
{'name': 'Andy', 'surname': 'Millers', 'amount': '947.76', 'delta_t': '3.47', 'flag': 0}
{'name': 'Andy', 'surname': 'Smith', 'amount': '866.35', 'delta_t': '4.92', 'flag': 0}
{'name': 'Alice', 'surname': 'Jones', 'amount': '942.77', 'delta_t': '2.70', 'flag': 0}
{'name': 'Joe', 'surname': 'Jones', 'amount': '964.01', 'delta_t': '8.62', 'flag': 1}
{'name': 'Andy', 'surname': 'Johnson', 'amount': '962.51', 'delta_t': '6.80', 'flag': 1}
{'name': 'Alice', 'surname': 'Johnson', 'amount': '919.19', 'delta_t': '1.22', 'flag': 0}
{'name': 'John', 'surname': 'Jones', 'amount': '259.72', 'delta_t': '0.27', 'flag': 1}
{'name': 'Andy', 'surname': 'Johnson', 'amount': '207.83', 'delta_t': '3.62', 'flag': 0}
{'name': 'Alice', 'surname': 'Smith', 'amount': '459.61', 'delta_t': '3.59', 'flag': 0}
...



-------------------------------------------
Time: 2022-05-17 13:11:54
-------------------------------------------
{'name': 'Andy', 'surname': 'Smith', 'amount': '650.06', 'delta_t': '3.88', 'flag': 0}
{'name': 'Andy', 'surname': 'Smith', 'amount': '742.05', 'delta_t': '7.48', 'flag': 0}
{'name': 'John', 'surname': 'Millers', 'amount': '482.91', 'delta_t': '1.82', 'flag': 0}
{'name': 'Andy', 'surname': 'Smith', 'amount': '839.49', 'delta_t': '3.09', 'flag': 0}
{'name': 'Alice', 'surname': 'Millers', 'amount': '613.95', 'delta_t': '0.23', 'flag': 1}
{'name': 'Alice', 'surname': 'Smith', 'amount': '70.54', 'delta_t': '9.42', 'flag': 0}
{'name': 'John', 'surname': 'Smith', 'amount': '209.50', 'delta_t': '5.46', 'flag': 0}
{'name': 'Alice', 'surname': 'Johnson', 'amount': '806.04', 'delta_t': '6.55', 'flag': 0}
{'name': 'Andy', 'surname': 'Johnson', 'amount': '309.49', 'delta_t': '3.37', 'flag': 0}
{'name': 'Andy', 'surname': 'Jones', 'amount': '989.39', 'delta_t': '8.31', 'flag': 0}
...



-------------------------------------------
Time: 2022-05-17 13:11:56
-------------------------------------------
{'name': 'Alice', 'surname': 'Millers', 'amount': '875.02', 'delta_t': '0.11', 'flag': 0}
{'name': 'Andy', 'surname': 'Jones', 'amount': '268.28', 'delta_t': '7.17', 'flag': 0}
{'name': 'Joe', 'surname': 'Smith', 'amount': '770.21', 'delta_t': '8.84', 'flag': 0}
{'name': 'Joe', 'surname': 'Millers', 'amount': '501.36', 'delta_t': '8.41', 'flag': 1}
{'name': 'Alice', 'surname': 'Jones', 'amount': '368.23', 'delta_t': '2.11', 'flag': 0}
{'name': 'Andy', 'surname': 'Jones', 'amount': '549.66', 'delta_t': '3.42', 'flag': 0}
{'name': 'Joe', 'surname': 'Millers', 'amount': '803.68', 'delta_t': '4.16', 'flag': 0}
{'name': 'Andy', 'surname': 'Smith', 'amount': '304.87', 'delta_t': '5.59', 'flag': 0}
{'name': 'Alice', 'surname': 'Johnson', 'amount': '301.27', 'delta_t': '8.17', 'flag': 0}
{'name': 'Joe', 'surname': 'Smith', 'amount': '195.65', 'delta_t': '1.72', 'flag': 0}
...



...

-------------------------------------------
Time: 2022-05-17 13:14:20
-------------------------------------------
{'name': 'John', 'surname': 'Smith', 'amount': '842.86', 'delta_t': '7.33', 'flag': 0}
{'name': 'John', 'surname': 'Jones', 'amount': '431.42', 'delta_t': '7.12', 'flag': 1}
{'name': 'Joe', 'surname': 'Jones', 'amount': '234.84', 'delta_t': '9.63', 'flag': 0}
{'name': 'Alice', 'surname': 'Jones', 'amount': '10.99', 'delta_t': '8.00', 'flag': 1}
{'name': 'Alice', 'surname': 'Smith', 'amount': '440.26', 'delta_t': '1.18', 'flag': 0}
{'name': 'Andy', 'surname': 'Millers', 'amount': '67.64', 'delta_t': '2.36', 'flag': 0}
{'name': 'Andy', 'surname': 'Jones', 'amount': '691.52', 'delta_t': '8.21', 'flag': 1}
{'name': 'John', 'surname': 'Smith', 'amount': '496.05', 'delta_t': '5.59', 'flag': 0}
{'name': 'Alice', 'surname': 'Millers', 'amount': '358.05', 'delta_t': '8.66', 'flag': 0}
{'name': 'Alice', 'surname': 'Jones', 'amount': '843.23', 'delta_t': '8.95', 'flag': 0}
...



-------------------------------------------
Time: 2022-05-17 13:14:22
-------------------------------------------
{'name': 'Alice', 'surname': 'Smith', 'amount': '233.09', 'delta_t': '4.51', 'flag': 1}
{'name': 'Alice', 'surname': 'Johnson', 'amount': '122.54', 'delta_t': '9.04', 'flag': 0}
{'name': 'Alice', 'surname': 'Millers', 'amount': '710.26', 'delta_t': '0.61', 'flag': 0}
{'name': 'Alice', 'surname': 'Johnson', 'amount': '220.08', 'delta_t': '9.81', 'flag': 0}
{'name': 'Alice', 'surname': 'Johnson', 'amount': '933.83', 'delta_t': '7.85', 'flag': 0}
{'name': 'John', 'surname': 'Johnson', 'amount': '997.50', 'delta_t': '5.27', 'flag': 0}
{'name': 'John', 'surname': 'Johnson', 'amount': '931.36', 'delta_t': '0.92', 'flag': 0}
{'name': 'Andy', 'surname': 'Johnson', 'amount': '480.02', 'delta_t': '5.10', 'flag': 0}
{'name': 'Joe', 'surname': 'Jones', 'amount': '190.42', 'delta_t': '7.42', 'flag': 1}
{'name': 'Andy', 'surname': 'Millers', 'amount': '899.30', 'delta_t': '2.81', 'flag':

-------------------------------------------
Time: 2022-05-17 13:14:24
-------------------------------------------
{'name': 'Alice', 'surname': 'Jones', 'amount': '931.81', 'delta_t': '7.69', 'flag': 0}
{'name': 'Alice', 'surname': 'Johnson', 'amount': '958.07', 'delta_t': '2.91', 'flag': 0}
{'name': 'Joe', 'surname': 'Jones', 'amount': '97.78', 'delta_t': '2.66', 'flag': 0}
{'name': 'John', 'surname': 'Smith', 'amount': '5.72', 'delta_t': '3.71', 'flag': 0}
{'name': 'Andy', 'surname': 'Smith', 'amount': '542.80', 'delta_t': '0.32', 'flag': 0}
{'name': 'John', 'surname': 'Johnson', 'amount': '361.76', 'delta_t': '7.19', 'flag': 0}
{'name': 'Andy', 'surname': 'Smith', 'amount': '789.88', 'delta_t': '9.68', 'flag': 0}
{'name': 'Joe', 'surname': 'Smith', 'amount': '424.46', 'delta_t': '1.07', 'flag': 0}
{'name': 'Alice', 'surname': 'Millers', 'amount': '473.33', 'delta_t': '0.38', 'flag': 1}
{'name': 'Joe', 'surname': 'Johnson', 'amount': '206.98', 'delta_t': '4.36', 'flag': 0}
...



-------------------------------------------
Time: 2022-05-17 13:14:26
-------------------------------------------
{'name': 'Joe', 'surname': 'Smith', 'amount': '985.99', 'delta_t': '6.46', 'flag': 0}
{'name': 'Alice', 'surname': 'Millers', 'amount': '119.27', 'delta_t': '2.49', 'flag': 0}
{'name': 'Joe', 'surname': 'Johnson', 'amount': '137.20', 'delta_t': '0.11', 'flag': 1}
{'name': 'John', 'surname': 'Smith', 'amount': '792.63', 'delta_t': '9.68', 'flag': 0}
{'name': 'Alice', 'surname': 'Johnson', 'amount': '21.75', 'delta_t': '2.49', 'flag': 0}
{'name': 'Alice', 'surname': 'Jones', 'amount': '488.26', 'delta_t': '4.46', 'flag': 0}
{'name': 'Andy', 'surname': 'Millers', 'amount': '916.80', 'delta_t': '2.16', 'flag': 0}
{'name': 'John', 'surname': 'Jones', 'amount': '633.39', 'delta_t': '4.29', 'flag': 1}
{'name': 'Andy', 'surname': 'Smith', 'amount': '906.91', 'delta_t': '8.31', 'flag': 1}
{'name': 'Alice', 'surname': 'Millers', 'amount': '265.96', 'delta_t': '10.00', 'flag': 0}
...


-------------------------------------------
Time: 2022-05-17 13:14:28
-------------------------------------------
{'name': 'John', 'surname': 'Johnson', 'amount': '148.62', 'delta_t': '3.20', 'flag': 0}
{'name': 'John', 'surname': 'Millers', 'amount': '876.58', 'delta_t': '4.58', 'flag': 0}
{'name': 'Alice', 'surname': 'Johnson', 'amount': '454.61', 'delta_t': '7.22', 'flag': 0}
{'name': 'John', 'surname': 'Smith', 'amount': '258.64', 'delta_t': '0.90', 'flag': 1}
{'name': 'Andy', 'surname': 'Smith', 'amount': '776.44', 'delta_t': '3.87', 'flag': 0}
{'name': 'Alice', 'surname': 'Jones', 'amount': '960.35', 'delta_t': '0.67', 'flag': 0}
{'name': 'John', 'surname': 'Smith', 'amount': '878.45', 'delta_t': '8.88', 'flag': 0}
{'name': 'John', 'surname': 'Jones', 'amount': '821.54', 'delta_t': '3.89', 'flag': 0}
{'name': 'Andy', 'surname': 'Johnson', 'amount': '334.43', 'delta_t': '2.56', 'flag': 0}
{'name': 'Andy', 'surname': 'Johnson', 'amount': '127.73', 'delta_t': '3.26', 'flag': 0}
...


In [11]:
ssc.stop(stopSparkContext=False)

22/05/17 13:14:32 ERROR ReceiverTracker: Deregistered receiver for stream 0: Stopped by driver


-------------------------------------------
Time: 2022-05-17 13:14:30
-------------------------------------------
{'name': 'Alice', 'surname': 'Johnson', 'amount': '885.67', 'delta_t': '6.89', 'flag': 0}
{'name': 'Alice', 'surname': 'Jones', 'amount': '711.12', 'delta_t': '8.61', 'flag': 0}
{'name': 'Joe', 'surname': 'Johnson', 'amount': '932.67', 'delta_t': '2.51', 'flag': 0}
{'name': 'Alice', 'surname': 'Jones', 'amount': '645.74', 'delta_t': '4.88', 'flag': 0}
{'name': 'Joe', 'surname': 'Jones', 'amount': '367.22', 'delta_t': '9.26', 'flag': 0}
{'name': 'John', 'surname': 'Jones', 'amount': '920.15', 'delta_t': '4.80', 'flag': 0}
{'name': 'Alice', 'surname': 'Johnson', 'amount': '946.23', 'delta_t': '7.00', 'flag': 0}
{'name': 'Andy', 'surname': 'Johnson', 'amount': '533.97', 'delta_t': '4.99', 'flag': 0}
{'name': 'Andy', 'surname': 'Jones', 'amount': '358.90', 'delta_t': '7.69', 'flag': 0}
{'name': 'Joe', 'surname': 'Smith', 'amount': '746.40', 'delta_t': '4.40', 'flag': 0}
...

--

22/05/17 13:14:34 ERROR TaskSchedulerImpl: Lost executor 1 on 172.21.0.4: Remote RPC client disassociated. Likely due to containers exceeding thresholds, or network issues. Check driver logs for WARN messages.


Now that we know we can stream data to Spark we would like to start performing basic operations in a distributed fashion.

Once the SparkStreamingContext is stopped we have to re-create a new one, as the connection between the socket and Spark is lost.

Before running the following cells we should therefore:
1. create a new `scc` object
2. point it to the proper TCP socket and port
3. start again the python producer application

In [12]:
# create a new Spark StreamingContext with a batch wall-time of 2 seconds
ssc = StreamingContext(sc, 2)

In [13]:
# define the socket stream using the appropriate endpoint and port
socket_stream = ssc.socketTextStream(hostname, 5555)

#### start once again the python producer script

We now start listening on the TCP socket, interpreting the input data stream as json loads.

Remember to get rid of the `pprint()` action, that would otherwise be performed continously, dumping the input data into the Jupyter cells.

In [14]:
# create a new json_stream object by reading the json loads from the socket
json_stream = socket_stream.map(lambda msg: json.loads(msg))

We may want to convert each batch into a Spark DataFrame to have access to the higher level APIs. 

In order to do that, let's first convert all the numeric features of the json into python floats and integers variables. 
This is a simple type cast operation in python, that can be easily parallelized.

The python dictionary produced by each json message, re-casted with the proper data types, can then be used to create a `Row`for each transaction.

In [15]:
from pyspark.sql import Row

# create a row for each message 
#   convert each numerical value to the proper python type
#   create a row from each message
def create_row_rdd(t):
    t['amount'] = float(t['amount'])
    t['delta t'] = float(t['delta t'])
    t['flag'] = int(t['flag'])
    return Row(**t)

# apply the transformation to the json_stream rdd
row_stream = json_stream.map(create_row_rdd)

The method `DStream.foreachRDD` can be used to apply custom transformations to each *batch* of data. 

In our case, we are insterested in converting each batch of data into a Spark dataframe and perform operations, such as finding the number of transactions for each user. 

For the scope of this simple use-case we can consider that all batches where a user has performed more than one transaction with the `flag` field equal to one can be identified as fraudulent.

In [16]:
# if left unconstrained, Spark might want to create a very 
# large number of partitions for this streaming application
# 
# using way more partitions than necessary always results in
# a huge over-head due to the partition-to-partition communications
# 
# this line is a trick to force Spark to use a small number of partitions
# thus making it more efficient in the case of small workloads and few executors

spark.conf.set("spark.sql.shuffle.partitions", 4)

In [18]:
from pyspark.sql.functions import concat, col, lit, countDistinct

# process each bach to identify possibly fraudolent transactions
#   1. convert the rdd into a dataframe (provide the schema if necessary)
#   2. compute the number of transactions per batch per user (a unique combination of first_and_lastname)
#   3. identify all the "suspicios" transactions by counting the number of transactions from a unique user 
#   3. print the resulting dataframe to mimick a live reporting of the suspicious transactions

def process_batch(rdd):
    # convert rdd to df
    #   check the documentation and/or the Lecture2 notebook 
    #   for details on how to create and pass a schema to a dataframe   
    df = rdd.toDF(
        schema='name string, surname string, mount float, delta_t, float, flag_int'
    )
    
    # find number of transactions for each user when flag = 1 
    #    declare a new column to create a unique user identifier 
    #    this can be easily done by concatenating first- and last-name fields
    #    check the concat function from pyspark.sql.functions 
    num_transactions = (
        df
        .where(col('flag')==1)
        .withColumn('id', concat(col('name'), col('surname')))
        .groupBy('id')
        .count()
    )
    
    # find suspicious transactions
    #    filter only users with more than one transaction per batch
    #    create a "fraud" column with a value of 1 for the selected users (check the lit function)
    #    from the dataframe, project only the unique id and fraud columns
    sus_transactions = (
        num_transactions
        .where(col('count')>1)
        .withColumn('fraud', lit(1))
        .select(col('id'), col('fraud'))
    )
    
    # (trigger an automatic alert)
    # print the first 5 items of the resulting dataframe
    sus_transactions.show(5)

Finally, instruct Spark to execute this `process_batch` function for each RDD you will have in your DStream

In [19]:
row_stream.foreachRDD(process_batch)

Now you should be ready to start the spark streaming context

In [21]:
# start streaming context
ssc.start()

22/05/17 13:43:29 ERROR JobScheduler: Error running job streaming job 1652795006000 ms.0
org.apache.spark.SparkException: An exception was raised by Python:
Traceback (most recent call last):
  File "/usr/bin/spark-3.2.1-bin-hadoop3.2/python/pyspark/streaming/util.py", line 68, in call
    r = self.func(t, *rdds)
  File "/usr/bin/spark-3.2.1-bin-hadoop3.2/python/pyspark/streaming/dstream.py", line 155, in <lambda>
    func = lambda t, rdd: old_func(rdd)
  File "/tmp/ipykernel_11/1232840701.py", line 13, in process_batch
    df = rdd.toDF(
  File "/usr/bin/spark-3.2.1-bin-hadoop3.2/python/pyspark/sql/session.py", line 66, in toDF
    return sparkSession.createDataFrame(self, schema, sampleRatio)
  File "/usr/bin/spark-3.2.1-bin-hadoop3.2/python/pyspark/sql/session.py", line 661, in createDataFrame
    schema = _parse_datatype_string(schema)
  File "/usr/bin/spark-3.2.1-bin-hadoop3.2/python/pyspark/sql/types.py", line 843, in _parse_datatype_string
    raise e
  File "/usr/bin/spark-3.2.

22/05/17 13:43:30 ERROR JobScheduler: Error running job streaming job 1652795010000 ms.0
org.apache.spark.SparkException: An exception was raised by Python:
Traceback (most recent call last):
  File "/usr/bin/spark-3.2.1-bin-hadoop3.2/python/pyspark/streaming/util.py", line 68, in call
    r = self.func(t, *rdds)
  File "/usr/bin/spark-3.2.1-bin-hadoop3.2/python/pyspark/streaming/dstream.py", line 155, in <lambda>
    func = lambda t, rdd: old_func(rdd)
  File "/tmp/ipykernel_11/1232840701.py", line 13, in process_batch
    df = rdd.toDF(
  File "/usr/bin/spark-3.2.1-bin-hadoop3.2/python/pyspark/sql/session.py", line 66, in toDF
    return sparkSession.createDataFrame(self, schema, sampleRatio)
  File "/usr/bin/spark-3.2.1-bin-hadoop3.2/python/pyspark/sql/session.py", line 661, in createDataFrame
    schema = _parse_datatype_string(schema)
  File "/usr/bin/spark-3.2.1-bin-hadoop3.2/python/pyspark/sql/types.py", line 843, in _parse_datatype_string
    raise e
  File "/usr/bin/spark-3.2.

22/05/17 13:43:34 ERROR JobScheduler: Error running job streaming job 1652795014000 ms.0
org.apache.spark.SparkException: An exception was raised by Python:
Traceback (most recent call last):
  File "/usr/bin/spark-3.2.1-bin-hadoop3.2/python/pyspark/streaming/util.py", line 68, in call
    r = self.func(t, *rdds)
  File "/usr/bin/spark-3.2.1-bin-hadoop3.2/python/pyspark/streaming/dstream.py", line 155, in <lambda>
    func = lambda t, rdd: old_func(rdd)
  File "/tmp/ipykernel_11/1232840701.py", line 13, in process_batch
    df = rdd.toDF(
  File "/usr/bin/spark-3.2.1-bin-hadoop3.2/python/pyspark/sql/session.py", line 66, in toDF
    return sparkSession.createDataFrame(self, schema, sampleRatio)
  File "/usr/bin/spark-3.2.1-bin-hadoop3.2/python/pyspark/sql/session.py", line 661, in createDataFrame
    schema = _parse_datatype_string(schema)
  File "/usr/bin/spark-3.2.1-bin-hadoop3.2/python/pyspark/sql/types.py", line 843, in _parse_datatype_string
    raise e
  File "/usr/bin/spark-3.2.

22/05/17 13:43:38 ERROR JobScheduler: Error running job streaming job 1652795018000 ms.0
org.apache.spark.SparkException: An exception was raised by Python:
Traceback (most recent call last):
  File "/usr/bin/spark-3.2.1-bin-hadoop3.2/python/pyspark/streaming/util.py", line 68, in call
    r = self.func(t, *rdds)
  File "/usr/bin/spark-3.2.1-bin-hadoop3.2/python/pyspark/streaming/dstream.py", line 155, in <lambda>
    func = lambda t, rdd: old_func(rdd)
  File "/tmp/ipykernel_11/1232840701.py", line 13, in process_batch
    df = rdd.toDF(
  File "/usr/bin/spark-3.2.1-bin-hadoop3.2/python/pyspark/sql/session.py", line 66, in toDF
    return sparkSession.createDataFrame(self, schema, sampleRatio)
  File "/usr/bin/spark-3.2.1-bin-hadoop3.2/python/pyspark/sql/session.py", line 661, in createDataFrame
    schema = _parse_datatype_string(schema)
  File "/usr/bin/spark-3.2.1-bin-hadoop3.2/python/pyspark/sql/types.py", line 843, in _parse_datatype_string
    raise e
  File "/usr/bin/spark-3.2.

22/05/17 13:43:42 ERROR JobScheduler: Error running job streaming job 1652795022000 ms.0
org.apache.spark.SparkException: An exception was raised by Python:
Traceback (most recent call last):
  File "/usr/bin/spark-3.2.1-bin-hadoop3.2/python/pyspark/streaming/util.py", line 68, in call
    r = self.func(t, *rdds)
  File "/usr/bin/spark-3.2.1-bin-hadoop3.2/python/pyspark/streaming/dstream.py", line 155, in <lambda>
    func = lambda t, rdd: old_func(rdd)
  File "/tmp/ipykernel_11/1232840701.py", line 13, in process_batch
    df = rdd.toDF(
  File "/usr/bin/spark-3.2.1-bin-hadoop3.2/python/pyspark/sql/session.py", line 66, in toDF
    return sparkSession.createDataFrame(self, schema, sampleRatio)
  File "/usr/bin/spark-3.2.1-bin-hadoop3.2/python/pyspark/sql/session.py", line 661, in createDataFrame
    schema = _parse_datatype_string(schema)
  File "/usr/bin/spark-3.2.1-bin-hadoop3.2/python/pyspark/sql/types.py", line 843, in _parse_datatype_string
    raise e
  File "/usr/bin/spark-3.2.

...

22/05/17 13:46:22 ERROR JobScheduler: Error running job streaming job 1652795182000 ms.0
org.apache.spark.SparkException: An exception was raised by Python:
Traceback (most recent call last):
  File "/usr/bin/spark-3.2.1-bin-hadoop3.2/python/pyspark/streaming/util.py", line 68, in call
    r = self.func(t, *rdds)
  File "/usr/bin/spark-3.2.1-bin-hadoop3.2/python/pyspark/streaming/dstream.py", line 155, in <lambda>
    func = lambda t, rdd: old_func(rdd)
  File "/tmp/ipykernel_11/1232840701.py", line 13, in process_batch
    df = rdd.toDF(
  File "/usr/bin/spark-3.2.1-bin-hadoop3.2/python/pyspark/sql/session.py", line 66, in toDF
    return sparkSession.createDataFrame(self, schema, sampleRatio)
  File "/usr/bin/spark-3.2.1-bin-hadoop3.2/python/pyspark/sql/session.py", line 661, in createDataFrame
    schema = _parse_datatype_string(schema)
  File "/usr/bin/spark-3.2.1-bin-hadoop3.2/python/pyspark/sql/types.py", line 843, in _parse_datatype_string
    raise e
  File "/usr/bin/spark-3.2.

22/05/17 13:46:26 ERROR JobScheduler: Error running job streaming job 1652795186000 ms.0
org.apache.spark.SparkException: An exception was raised by Python:
Traceback (most recent call last):
  File "/usr/bin/spark-3.2.1-bin-hadoop3.2/python/pyspark/streaming/util.py", line 68, in call
    r = self.func(t, *rdds)
  File "/usr/bin/spark-3.2.1-bin-hadoop3.2/python/pyspark/streaming/dstream.py", line 155, in <lambda>
    func = lambda t, rdd: old_func(rdd)
  File "/tmp/ipykernel_11/1232840701.py", line 13, in process_batch
    df = rdd.toDF(
  File "/usr/bin/spark-3.2.1-bin-hadoop3.2/python/pyspark/sql/session.py", line 66, in toDF
    return sparkSession.createDataFrame(self, schema, sampleRatio)
  File "/usr/bin/spark-3.2.1-bin-hadoop3.2/python/pyspark/sql/session.py", line 661, in createDataFrame
    schema = _parse_datatype_string(schema)
  File "/usr/bin/spark-3.2.1-bin-hadoop3.2/python/pyspark/sql/types.py", line 843, in _parse_datatype_string
    raise e
  File "/usr/bin/spark-3.2.

22/05/17 13:46:30 ERROR JobScheduler: Error running job streaming job 1652795190000 ms.0
org.apache.spark.SparkException: An exception was raised by Python:
Traceback (most recent call last):
  File "/usr/bin/spark-3.2.1-bin-hadoop3.2/python/pyspark/streaming/util.py", line 68, in call
    r = self.func(t, *rdds)
  File "/usr/bin/spark-3.2.1-bin-hadoop3.2/python/pyspark/streaming/dstream.py", line 155, in <lambda>
    func = lambda t, rdd: old_func(rdd)
  File "/tmp/ipykernel_11/1232840701.py", line 13, in process_batch
    df = rdd.toDF(
  File "/usr/bin/spark-3.2.1-bin-hadoop3.2/python/pyspark/sql/session.py", line 66, in toDF
    return sparkSession.createDataFrame(self, schema, sampleRatio)
  File "/usr/bin/spark-3.2.1-bin-hadoop3.2/python/pyspark/sql/session.py", line 661, in createDataFrame
    schema = _parse_datatype_string(schema)
  File "/usr/bin/spark-3.2.1-bin-hadoop3.2/python/pyspark/sql/types.py", line 843, in _parse_datatype_string
    raise e
  File "/usr/bin/spark-3.2.

22/05/17 13:46:34 ERROR JobScheduler: Error running job streaming job 1652795194000 ms.0
org.apache.spark.SparkException: An exception was raised by Python:
Traceback (most recent call last):
  File "/usr/bin/spark-3.2.1-bin-hadoop3.2/python/pyspark/streaming/util.py", line 68, in call
    r = self.func(t, *rdds)
  File "/usr/bin/spark-3.2.1-bin-hadoop3.2/python/pyspark/streaming/dstream.py", line 155, in <lambda>
    func = lambda t, rdd: old_func(rdd)
  File "/tmp/ipykernel_11/1232840701.py", line 13, in process_batch
    df = rdd.toDF(
  File "/usr/bin/spark-3.2.1-bin-hadoop3.2/python/pyspark/sql/session.py", line 66, in toDF
    return sparkSession.createDataFrame(self, schema, sampleRatio)
  File "/usr/bin/spark-3.2.1-bin-hadoop3.2/python/pyspark/sql/session.py", line 661, in createDataFrame
    schema = _parse_datatype_string(schema)
  File "/usr/bin/spark-3.2.1-bin-hadoop3.2/python/pyspark/sql/types.py", line 843, in _parse_datatype_string
    raise e
  File "/usr/bin/spark-3.2.

22/05/17 13:46:38 ERROR JobScheduler: Error running job streaming job 1652795198000 ms.0
org.apache.spark.SparkException: An exception was raised by Python:
Traceback (most recent call last):
  File "/usr/bin/spark-3.2.1-bin-hadoop3.2/python/pyspark/streaming/util.py", line 68, in call
    r = self.func(t, *rdds)
  File "/usr/bin/spark-3.2.1-bin-hadoop3.2/python/pyspark/streaming/dstream.py", line 155, in <lambda>
    func = lambda t, rdd: old_func(rdd)
  File "/tmp/ipykernel_11/1232840701.py", line 13, in process_batch
    df = rdd.toDF(
  File "/usr/bin/spark-3.2.1-bin-hadoop3.2/python/pyspark/sql/session.py", line 66, in toDF
    return sparkSession.createDataFrame(self, schema, sampleRatio)
  File "/usr/bin/spark-3.2.1-bin-hadoop3.2/python/pyspark/sql/session.py", line 661, in createDataFrame
    schema = _parse_datatype_string(schema)
  File "/usr/bin/spark-3.2.1-bin-hadoop3.2/python/pyspark/sql/types.py", line 843, in _parse_datatype_string
    raise e
  File "/usr/bin/spark-3.2.

In [22]:
# stop streaming context - no not stop the SparkContext
ssc.stop(stopSparkContext=False)

22/05/17 13:46:38 ERROR ReceiverTracker: Deregistered receiver for stream 0: Stopped by driver
22/05/17 13:46:39 ERROR TaskSchedulerImpl: Lost executor 0 on 172.21.0.3: Remote RPC client disassociated. Likely due to containers exceeding thresholds, or network issues. Check driver logs for WARN messages.
22/05/17 13:46:39 WARN BlockManagerMasterEndpoint: No more replicas available for input-0-1652793270600 !
22/05/17 13:46:39 WARN BlockManagerMasterEndpoint: No more replicas available for input-0-1652793270800 !
22/05/17 13:46:39 WARN BlockManagerMasterEndpoint: No more replicas available for input-0-1652793269800 !


## Stop worker and master

In [ ]:
sc.stop()
spark.stop()

In [ ]:
%%script bash --no-raise-error

if [[ "$CLUSTER_TYPE" != "docker_cluster" ]]; then
    # stop worker 
    $SPARK_HOME/sbin/stop-worker.sh
    
    # start master
    $SPARK_HOME/sbin/stop-master.sh
fi